# Ranking Database Management Systems

In [ ]:
import requests
from retrying import retry
import logging
import pandas as pd
import numpy as np
import urllib2
import dill
import time
import json
from datetime import date, timedelta
from bs4 import BeautifulSoup
import bs4
import unicodedata
import csv
import sys
import get_stack_overflow_data as gso
from string import strip,split
sys.path.append('../code')
sys.path.append('../data')
import re

#function that adds a delay before running a function, to be used as a decorator.
def sleeper(secs):
    def realsleeper(func):
        def wrapper(*args,**kwargs):
            time.sleep(secs)
            return func(*args,**kwargs)
        return wrapper
    return realsleeper


## Create list of database management systems

### 1 https://db-engines.com/en/ranking

In [ ]:
r1 = requests.get('https://db-engines.com/en/ranking')
soup1 = BeautifulSoup(r1.text, 'lxml')
db_list1 = []
for atag in soup1.findAll('a', attrs={'class':None}):
    if (type(atag.contents[0]) == bs4.element.NavigableString):
        print atag.contents[0]
        db_list1.append(atag.contents[0])
    else:
        db_list1.append('')

In [ ]:
#CODE FOR CLEANING UP db_list1

#Set of extraneous items to remove from the list:
items_to_remove1 = set(['Relational DBMS','Document store','Key-value store','Search engine','Wide column store','Multi-model','Graph DBMS','Time Series DBMS','Content store','Navigational DBMS','Object oriented DBMS','Native XML DBMS','Event Store','RDF store'])

db_set1 = set(db_list1[45:697]) #a set used in cleaning up db_list1. As of 3-25-18, the index range [48:697] was the correct one to use. This may change if and when the ranking list is updated.
db_set1.difference_update(items_to_remove1)

#opensource(db_name) returns True if db_name is an open source database engine, false otherwise
def opensource(db_name):
    url = 'https://db-engines.com/en/system/' + db_name
     
    url = url.replace('\\x','%')

    r = requests.get(url)

    if re.search('>Open Source',r.text):
        return True
    else:
        return False

#remove databases engines that are not open source
db_list1 = [db_name for db_name in db_list1 if opensource(db_name)]

#encode as utf-8, make lowercase, and remove whitespace
db_list1 = [strip(x).encode('utf-8').lower() for x in db_list1]

In [ ]:
print len(db_list1)
print sorted(db_list1)

### 2 https://en.wikipedia.org/wiki/List_of_relational_database_management_systems

In [ ]:
r1 = requests.get('https://en.wikipedia.org/wiki/List_of_relational_database_management_systems')
soup2 = BeautifulSoup(r1.text, 'lxml')
db_list2 = []
for litags in soup2.findAll('li', attrs={'class':None}):
    try:
        for atag in litags.find('a'):
            db_list2.append(atag.encode('utf-8').lower())
    except:
        pass

In [ ]:
#Use split to remove expressions in parentheses, which are notes about the items in the list.
#As of 3-25-18, the index range [:96] was the correct one to use. This may change if and when the list is updated.
db_list2 = [split(x, r' (')[0] for x in db_list2[:96]]

In [ ]:
print len(db_list2)
print len(filter(lambda x:(x not in db_list1),db_list2))
print filter(lambda x:(x not in db_list1),db_list2)

In [ ]:
#remove items in db_list2 that are duplicates of items in db_list1 by a different name (e.g. 4d vs. 4th dimension)
print len(db_list2)
db_list2_additional_duplicates = ['4th dimension','adabas d','apache derby','ca datacom','ca idms','empress embedded database','exasolution','filemaker pro','sql azure','openlink virtuoso','openlink virtuoso universal server','postgres plus advanced server','progress software','sap adaptive server enterprise','sql anywhere','unidata','universe']
db_list2 = filter(lambda x:(x not in db_list2_additional_duplicates),db_list2)
print len(db_list2)

#remove items in db_list2 that are not open-source

db_list2_not_open_source = ['alpha five',
'aster data',
'clarion',
'grovesite',
'helix database',
'ibm db2',
'ibm lotus approach',
'ibm db2 express-c',
'intersystems cach\xc3\xa9',
'microsoft jet database engine',
'microsoft sql server express',
'microsoft visual foxpro',
'omnis studio',
'panorama',
'pervasive psql',
'polyhedra',
'rdm server', 
'sand cdbms', 
'unisys rdms 2200',
'vectorwise']

db_list2 = filter(lambda x:(x not in db_list2_not_open_source),db_list2)

print len(db_list2)


In [ ]:
db_list12 = list(set(db_list1 + db_list2))
print(sorted(db_list12))

### 3 https://www.quora.com/Which-is-the-best-database-for-big-data

In [ ]:
#items entered manually from this Quora discussion (only entering items that are open-source)
db_list3 = ['hive','hbase','apache phoenix','mongodb','druid','mapd','couchdb','hbase','riak','zookeeper','cassandra','voldemort']

print len(db_list3)
db_list3_additional_duplicates = ['riak','voldemort']
db_list3 = filter(lambda x:(x not in db_list3_additional_duplicates),db_list3)
print len(db_list3)

### combine and edit lists

In [ ]:
db_list = list(set(db_list1 + db_list2 + db_list3))

In [ ]:
#add oracle to db_list, since it is listed under different names
#db_list.append('oracle')

In [ ]:
print len(db_list)
print sorted(db_list)

In [ ]:
with open('db_list.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(db_list)

In [ ]:
#remove database engines that are proprietary with no github repository
db_list.remove('microsoft access')
db_list.remove('microsoft sql server')
db_list.remove('oracle')
db_list.remove('oracle nosql')
db_list.remove('oracle rdb')
db_list.remove('clustrix')
db_list.remove('enterprisedb')
db_list.remove('infobright')
db_list.remove('linter')
db_list.remove('maxdb')
db_list.remove('memsql')
db_list.remove('mimer sql')
db_list.remove('nexusdb')
db_list.remove('nonstop sql')
db_list.remove('nuodb')
db_list.remove('openbase')
db_list.remove('r:base')
db_list.remove('rdm embedded')
db_list.remove('sap hana')
db_list.remove('sap iq')
db_list.remove('soliddb')
db_list.remove('splice machine')
db_list.remove('sqlbase')
db_list.remove('sqream db')
db_list.remove('tibero')
db_list.remove('timesten')



In [ ]:
with open("../data/DB_final.txt", "w") as f:
    f.writelines(" ".join(db_list))

In [ ]:
db_list.sort()
print db_list

### get metrics

In [ ]:
logging.basicConfig(level=logging.INFO)

### github stats

In [ ]:
%with open("../code/secrets/github-token.nogit", "rb") as f:
    token = f.read()

headers = {'Authorization': 'token %s' % token}

@sleeper(3)#github will temporarily block requests from a user that makes more than 30 requests within a 60 second period. To be safe, use a 3 second pause between requests to limit rate to 20 requests per minute.
@retry(wait_exponential_multiplier=3000,wait_exponential_max=12000,stop_max_attempt_number=3)#in case request fails, retry up to 3 times, starting with a 3 second wait period and doubling that period each time.
def get_data_from_search_helper(query):
    r = requests.get('https://api.github.com/search/repositories?q='+\
                             query, 
                     headers=headers)
    return r

def get_data_from_search(query):
    """Use github search to return stars, forks for top query result"""
    
    r = get_data_from_search_helper(query)
    #r.raise_for_status()
    try:
        res = r.json()['items'][0]
        return {'toolkit': query, 'full_name': res['full_name'],
                'stars': int(res['stargazers_count']), 'forks': int(res['forks_count'])}
    except:
        return None

In [ ]:
data = [res for res in (get_data_from_search(q) for q in db_list)
        if res is not None]

In [ ]:
github = pd.DataFrame(data)[['toolkit', 'full_name', 'forks', 'stars']]

In [ ]:
github.shape

In [ ]:
with pd.option_context('display.width', 160, 'display.max_rows', None,'display.max_columns', None):
    print(github)

In [ ]:
#github search returned wrong repo for many results, so they are manually changed below

corrections = [
    ('aerospike','aerospike'),
    ('boltdb','boltdb/bolt'),
    ('couchbase','couchbase'),
    ('cratedb','crate/crate'),
    ('csql',None),
    ('cubicweb',None),
    ('database management library',None),
    ('dataease',None),
    ('db4o',None),
    ('dbase',None),
    ('djondb',None),
    ('ehcache','ehcache'),
    ('exist-db','eXist-db/exist'),
    ('extremedb',None),
    ('frontbase',None),
    ('google fusion tables',None),
    ('grakn.ai','graknlabs/grakn'),
    ('graphite','graphite-project'),
    ('griddb','griddb_nosql'),
    ('h2','h2database'),
    ('hsqldb',None),
    ('iboxdb','iboxdb'),
    ('ignite','apache/ignite'),
    ('impala','apache/impala'),
    ('infogrid','infogrid-org'),
    ('informix',None),
    ('interbase',None),
    ('kyoto cabinet','alticelabs/kyoto/kyotocabinet'),
    ('kyoto tycoon','alticelabs/kyoto/kyototycoon'),
    ('libreoffice base',None),
    ('mapd','mapd/mapd-core'),
    ('mariadb','MariaDB'),
    ('mongodb','mongodb/mongo'),
    ('mysql','mysql'),
    ('netezza',None),
    ('openoffice.org base',None),
    ('oracle berkeley db',None),
    ('percona server for mongodb','percona/percona-server-mongodb'),
    ('percona server for mysql','percona/percona-server'),
    ('perst',None),
    ('postgresql','postgres/postgres'),
    ('prestodb','prestodb/presto'),
    ('project voldemort','voldemort/voldemort'),
    ('pyrrho',None),
    ('redland',None),
    ('resin cache',None),
    ('riak ts','basho/riak'),
    ('sap hana',None),
    ('scidb',None),
    ('scylladb','scylladb/scylla'),
    ('sedna','sedna/sedna'),
    ('senseidb','senseidb'),
    ('smallsql','kevinvandervlist/SE/tree/master/smallsql'),
    ('solr','apache/lucene-solr'),
    ('spark sql','apache/spark/tree/master/sql'),
    ('sqlite',None),
    ('teradata','teradata'),
    ('tinkergraph',None),
    ('tokyo cabinet',None),
    ('tokyo tyrant',None),
    ('torodb','torodb/torodb'),
    ('txtsql',None),
    ('vertica','vertica'),
    ('virtuoso','openlink/virtuoso-opensource'),
    ('wakandadb','WakandaDB'),
    ('Zookeeper','apache/zookeeper')
]

for toolkit,full_name in corrections:
    try:
        r = requests.get('https://api.github.com/repos/' + full_name, headers=headers)
        res = r.json()
        github.loc[github['toolkit'] == toolkit, 'full_name'] = full_name
        github.loc[github['toolkit'] == toolkit, 'forks'] = res['forks_count']
        github.loc[github['toolkit'] == toolkit, 'stars'] = res['stargazers_count']
    except:#to handle None
        github.loc[github['toolkit'] == toolkit, 'full_name'] = None
        github.loc[github['toolkit'] == toolkit, 'forks'] = 0
        github.loc[github['toolkit'] == toolkit, 'stars'] = 0       


In [ ]:
github.sort_values(['stars'], ascending=False).head(100)

In [ ]:
github.to_csv("../data/DB_results_github.csv", index=False)

### stackoverflow stats

In [ ]:
tag_list = [toolkit.replace(' ','-').replace(':','-') for toolkit in db_list]

In [ ]:
len(tag_list)

In [ ]:
#function for getting stack overflow tag counts
@sleeper(3)#attempt to avoid throttling
@retry(wait_exponential_multiplier=60000,wait_exponential_max=240000,stop_max_attempt_number=3)#in case request fails, retry up to 3 times, starting with a 1 minute wait period and doubling that period each time.
def tag_counts_builder_helper(list_to_build,list_entry):
    list_to_build += gso.get_tag_counts([list_entry])

In [ ]:
#build list of tag counts
tag_counts = []
for x in tag_list:
    try:
        tag_counts_builder_helper(tag_counts,x)
    except:
        tag_counts += [{'count':0,'has_synonyms':False,'is_moderator_only':False,'is_required':False,'name':x}]
        print 'exception'

In [ ]:
df_tags = pd.DataFrame(tag_counts)[['name', 'count']]

In [ ]:
df_tags.columns = ['toolkit', 'so_tag_counts']
df_tags

In [ ]:
@sleeper(3)#attempt to avoid throttling
@retry(wait_exponential_multiplier=60000,wait_exponential_max=240000,stop_max_attempt_number=3)#in case request fails, retry up to 3 times, starting with a 1 minute wait period and doubling that period each time.
#function used in building a dict each of whose values is the body count of the corresponding key
def body_counts_builder_helper(dict_to_build,key):
    dict_to_build[key] = gso.get_body_count([key])

In [ ]:
#build dict of body counts

body_counts = {}

for x in tag_list:
    body_counts_builder_helper(body_counts,x)

In [ ]:
df_questions = pd.DataFrame.from_dict(body_counts, orient='index')
df_questions.reset_index(inplace=True)
df_questions.columns = ['toolkit', 'so_question_count']

In [ ]:
so = df_tags.merge(df_questions, on='toolkit', how='outer')

In [ ]:
so['toolkit'] = so['toolkit'].apply(lambda x: str(x).replace('-',' '))
so.sort_values(['so_tag_counts'], ascending=False).head()

In [ ]:
so.to_csv("../data/DB_results_stackoverflow.csv", index=False)

### google search results stats

In [ ]:
from googleapiclient.discovery import build

In [ ]:
with open("../code/secrets/google_token.nogit", "rb") as f:
    my_api_key = f.read()
    
with open("../code/secrets/cse_token.nogit", "rb") as f:
    my_cse_id = f.read()

In [ ]:
def search_term_modifier(search_term):
    #replace space with +, indicating to search for both words:
    search_term = search_term.replace(' ','+')
    return search_term

In [ ]:
#function for returning the number of google search results
def google_search_results_count(search_term, api_key, cse_id):
    toolkit = search_term
    search_term = search_term_modifier(search_term)
    r= requests.get('https://www.googleapis.com/customsearch/v1?q="database"&alt=json&cx='+
                    my_cse_id+'&c2coff=1&dateRestrict=y5&exactTerms='+search_term+'&rc=1&key='+my_api_key)
    res = r.json()['queries']['request'][0]
    return {'toolkit': toolkit, 'search_results': int(res['totalResults'])}

#function for returning the rate of growth of google search results
def google_quarterly_growth_rate(search_term, api_key, cse_id):
    toolkit = search_term
    search_term = search_term_modifier(search_term)    
    ##get count for last 6 months--- dateRestrict=m6
    r= requests.get('https://www.googleapis.com/customsearch/v1?q="database"&alt=json&cx='+
                    my_cse_id+'&c2coff=1&dateRestrict=m6&exactTerms='+search_term+'&rc=1&key='+my_api_key)
    res = r.json()['queries']['request'][0]
    six_months = int(res['totalResults'])
    ##get count for last 3 months--- dateRestrict=m3    
    r= requests.get('https://www.googleapis.com/customsearch/v1?q="database"&alt=json&cx='+
                    my_cse_id+'&c2coff=1&dateRestrict=m3&exactTerms='+search_term+'&rc=1&key='+my_api_key)
    res = r.json()['queries']['request'][0]
    current_quarter = int(res['totalResults'])
    
    last_quarter = six_months - current_quarter
    if (last_quarter == 0):#for handling the divide by 0 case
        growth_rate = float('NaN')
    else:
        growth_rate = (float(current_quarter)-float(last_quarter))/float(last_quarter)
    return {'toolkit': toolkit, 'growth_rate': growth_rate}

In [ ]:
@sleeper(2)#attempt to avoid throttling
@retry(wait_exponential_multiplier=2000,wait_exponential_max=8000,stop_max_attempt_number=3)#in case request fails, retry up to 3 times, starting with a 2 second period and doubling that period each time.
#function used in building a list each of whose values is the google search results count
def google_results_builder_helper(list_to_build,query):
    res = google_search_results_count(query, my_api_key, my_cse_id)
    if res is not None:
        list_to_build.append(res)

In [ ]:
results = []
for q in db_list:
    google_results_builder_helper(results,q)

In [ ]:
resultsDF = pd.DataFrame(results)[['toolkit', 'search_results']]

In [ ]:
resultsDF.sort_values(['search_results'], ascending=False).head()

In [ ]:
@sleeper(8)#attempt to avoid throttling
@retry(wait_exponential_multiplier=8000,wait_exponential_max=32000,stop_max_attempt_number=3)#in case request fails, retry up to 3 times, starting with a 2 second period and doubling that period each time.
#function used in building a list each of whose values is the google quarterly growth rate
def growth_rate_builder_helper(list_to_build,query):
    res = google_quarterly_growth_rate(query, my_api_key, my_cse_id)
    if res is not None:
        list_to_build.append(res)

In [ ]:
growth_rate = []
for q in db_list:
    growth_rate_builder_helper(growth_rate,q)

In [ ]:
growthDF =  pd.DataFrame(growth_rate)[['toolkit', 'growth_rate']]

In [ ]:
growthDF.sort_values(['growth_rate'], ascending=False).head()

In [ ]:
googleDF = growthDF.merge(resultsDF, on='toolkit', copy = False)

In [ ]:
googleDF.to_csv("../data/DB_results_google.csv", index=False)

## Combine all data

In [ ]:
dltkDF = github.merge(so, on='toolkit', copy = False)
dltkDF = dltkDF.merge(googleDF, on='toolkit', copy = False)

In [ ]:
dltkDF.head()

In [ ]:
dltkDF.to_csv("../output/DB_data.csv", index=False)